In [ ]:
import matplotlib.pyplot as plot
import numpy
from numpy import random
from numpy import dot
from numpy import exp
from numpy import sum
from numpy import log
from numpy import zeros
from numpy import float64
from numpy import int64
from numpy import unravel_index
from numpy import max
from numpy import argmax
from numpy import copy
from numpy import save
from numpy import load
from numpy import array
from tqdm import tqdm
import pickle
from tabulate import tabulate

In [ ]:
def cross_entropy(outputs):
    """
    Fungsi cross entropy untuk mendapatkan nilai loss dari nilai hasil deteksi.

    :param outputs: nilai hasil deteksi.
    :return: nilai loss hasil deteksi.
    """

    loss_value = -log(outputs)

    return loss_value

def relu_activation(summa_nrc):
    """
    Fungsi aktivasi Rectified Linear Unit (ReLU) untuk melakukan aktifasi pada nilai input.

    :param summa_nrc: hasil operasi konvolusi untuk diaktifkan.
    :return: nilai hasil aktivasi ReLU.
    """

    if summa_nrc > 0:
        return summa_nrc
    else:
        return 0

def derivative_relu(summa_nrc):
    """
    Untuk mencari nilai turunan dari fungsi aktivasi ReLU.

    :param summa_nrc: nilai hasil aktivasi ReLU.
    :return: nilai hasil turunan ReLU.
    """

    if summa_nrc > 0:
        return 1
    else:
        return 0


def softmax_activation(summa):
    """
    Fungsi aktivasi softmax untuk melakukan aktivasi nilai dari fully connected layer ke dalam bentuk probabilitas.

    :param summa: nilai pembobotan di fully connected layer.
    :return: nilai hasil aktivasi dalam bentuk probabilitas.
    """

    numerator = exp(summa)
    denominator = sum(numerator)
    softmax_value = numerator / denominator

    return softmax_value

In [ ]:
class ConfusionMatrix:
    """
    Kelas untuk membuat confusion matrix dari performa model.

    Attributes:

    actual: 
        label/kelas sebenarnya dari citra.
    result: 
        label/kelas citra hasil deteksi.
    classes: 
        representasi kelas output.
    w_mask:
        representasi kelas tidak bermasker(0).
    mask:
        representasi kelas bermasker(1).
    """

    def __init__(self, actual_class, result_class):

        self.actual = actual_class  # Dalam bentuk array 1D
        self.result = result_class  # Dalam bentuk array 1D
        
        # Classes: 0 (Tidak bermasker), 1 (bermasker)
        self.classes = [0, 1]
        self.w_mask = self.classes[0]
        self.mask = self.classes[1]

    def true_negative(self):
        """
        Fungsi untuk menghitung nilai true negative.

        :returns: nilai true negative.
        """

        actual = self.actual
        result = self.result

        # Nilai awal true negative (tn)
        tn = 0

        for act_, res_ in zip(actual, result):
            if act_ == self.w_mask and res_ == self.w_mask:
                tn += 1
            else: pass

        return tn

    def false_positive(self):
        """
        Fungsi untuk menghitung nilai false positive.

        :returns: nilai false positive.
        """

        actual = self.actual
        result = self.result
        fp = 0

        for act_, res_ in zip(actual, result):
            if act_ == self.w_mask and res_ == self.mask:
                fp += 1
            else: pass

        return fp

    def false_negative(self):
        """
        Fungsi untuk menghitung nilai false negative.

        :returns: nilai false negative.
        """

        actual = self.actual
        result = self.result
        fn = 0

        for act_, res_ in zip(actual, result):
            if act_ == self.mask and res_ ==self.w_mask:
                fn += 1
            else: pass

        return fn

    def true_positive(self):
        """
        Fungsi untuk menghitung nilai true positive.

        :returns: nilai true positive.
        """

        actual = self.actual
        result = self.result
        tp = 0

        for act_, res_ in zip(actual, result):
            if act_ == self.mask and res_ == self.mask:
                tp += 1
            else: pass

        return tp

    def recall(self):
        """
        Fungsi untuk menghitung nilai recall.

        :returns: nilai recall.
        """
        tp = self.true_positive()
        fn = self.false_negative()

        recall = tp / (fn + tp)

        return recall

    def precision(self):
        """
        Fungsi untuk menghitung nilai recall.

        :returns: nilai recall.
        """
        tp = self.true_positive()
        fp = self.false_positive()

        precision = tp / (fp + tp)

        return precision

    def f1_score(self):
        """
        Fungsi untuk menghitung nilai f1-score.

        :returns: nilai f1-score.
        """

        precision = self.precision()
        recall = self.recall()

        f1_score = (2 * precision * recall) / (precision + recall)

        return f1_score

    def display_table(self):
        """
        Fungsi untuk menampilkan visualisasi tabel confusion matrix.

        :returns: tabel confusion matrix.
        """

        tn = self.true_negative()
        fp = self.false_positive()
        fn = self.false_negative()
        tp = self.true_positive()

        header_table = ["Actual/Result", "0", "1"]
        table = [["0", tn, fp], ["1", fn, tp]]
        tformat = "grid"

        return print(tabulate(tabular_data=table, headers=header_table,tablefmt=tformat))

    def accuracy(self):
        """
        Fungsi untuk menghitung nilai accuracy.

        :returns: nilai accuracy.
        """

        actual = self.actual
        result = self.result

        acc_ = 0
        n_data = len(result)

        for act_, res_ in zip(actual, result):
            if act_ == res_:
                acc_ += 1
            else:
                acc_ += 0

        accuracy = acc_ / n_data

        return accuracy

    def model_performance_graph(self):
        """
        Fungsi untuk menampilkan performa model dalam graph bar.

        :returns: graph bar.
        """

        actual = self.actual
        result = self.result

        accuracy = self.accuracy()
        recall = self.recall()
        precision = self.precision()
        f1_score = self.f1_score()

        data = {'accuracy':accuracy, 'racall':recall, 'precision': precision, 'f1_score':f1_score}

        performance = list(data.keys())
        values = list(data.values())

        # creating the bar plot
        plot.figure(figsize = (10, 5))
        plot.bar(performance, values, color ='gray',width = 0.4)
        plot.show()

        return

In [ ]:
class Conv2D:
    """
    Kelas untuk membuat convolution layer pertama.
    Satu citra input, dengan kernel 3 dimensi (n_kernel, kernel_size, kernel_size).

    Attributes:

    kernel: n-dimensional array.
        kernel yang digunakan pada layer ini.
    bias: 1-dimensional array.
        bias yang digunakan untuk layer ini.
    padding: int.
        jumlah padding yang digunakan di layer ini, nilai default=0.
    stride: int.
        jumlah perpindahan piksel untuk melakukan uperasi konvolusi, nilai default=1.
    """

    def __init__(self, kernel, bias, padding = 0, stride = 1):
        self.kernel = kernel
        self.bias = bias
        self.padding = padding
        self.stride = stride

    def forwardpass(self, image):
        """
        Fungsi untuk convolution layer ini melakukan forwardpass.

        :param image: citra input 2-dimensi (width, height).
        :return: hasil operasi konvolusi yang telah aktivasi relu, berbentuk n-dimensional array.
        """

        # Dapatkan jumlah dan ukuran kernel yang digunakan
        n_kernel, kernel_size, _ = self.kernel.shape

        # Menghitung ukuran output salah satu sisi (width dan height berukuran sama)
        cl_size = int((image.shape[0] - kernel_size + 2 * self.padding) / self.stride) + 1

        # Inisialisasi cl dengan dimensinya, nilai 0 untuk seluruh piksel
        cl = zeros((n_kernel, cl_size, cl_size), dtype=float64)

        # Inisialisasi output turunan fungsi relu, nilai 0 seluruh piksel
        d_cl_summa = zeros(cl.shape, dtype=float64)

        for nk in range(n_kernel):
            for row in range(cl_size):
                for col in range(cl_size):
                    # Mendapatkan lokasi region untuk operasi konvolusi
                    local_region = image[row:(row + kernel_size), col:(col + kernel_size)]

                    # Operasi konvolusi
                    summa_nrc = sum(local_region * self.kernel[nk]) + self.bias[nk]

                    # Operasi aktivasi relu
                    cl[nk, row, col] = relu_activation(summa_nrc)

                    # Operasi turunan relu
                    d_cl_summa[nk, row, col] = derivative_relu(cl[nk, row, col])

        return cl, d_cl_summa

In [ ]:
class Conv3D:
    """
    Kelas untuk membuat convolution layer kedua dan seterusnya.
    Dua atau lebih citra input, dengan kernel 3 dimensi (n_kernel, kernel_size, kernel_size).

    Attributes:

    kernel: n-dimensional array.
        Kernel yang digunakan pada layer ini.
    bias: 1-dimensional array.
        bias yang digunakan untuk layer ini.
    padding: int.
        jumlah padding yang digunakan di layer ini, nilai default=0.
    stride: int.
        jumlah perpindahan piksel untuk melakukan uperasi konvolusi, nilai default=1.
    """

    def __init__(self, kernel, bias, padding=0, stride=1):
        self.kernel = kernel
        self.bias = bias
        self.padding = padding
        self.stride = stride

    def forwardpass(self, pl):
        """
        Fungsi untuk convolutional layer ini melakukan forwardpass.

        :param pl: Hasil operasi pooling sebelumnya.
        :return: Hasil operasi konvolusi yang telah diaktifasi relu, berbentuk n-dimensional array.
        """

        # Mendapatkan jumlah kernel sekarang, sebelumnya, dan ukuran kernel dari dimensi kernel yang digunakan saat ini
        n_kernel_current, n_kernel_prev, kernel_size, _ = self.kernel.shape

        # Menghitung ukuran output salah satu sisi (width dan height berukuran sama)
        cl_size = int((pl.shape[1] - kernel_size + 2 * self.padding) / self.stride) + 1

        # Inisialisasi cl dan memberikan nilai 0 untuk seluruh piksel
        cl = zeros((n_kernel_current, cl_size, cl_size), dtype=float64)

        # Inisialisasi output turunan aktivasi relu dan memberikan nilai 0
        d_cl_summa = zeros(cl.shape, dtype=float64)

        d_summa_pl = zeros(pl.shape + cl.shape, dtype=float64)

        for nkc in range(n_kernel_current):
            for row in range(cl_size):
                for col in range(cl_size):
                    # Mendapatkan lokasi region untuk operasi konvolusi
                    local_region = pl[0:n_kernel_prev, row:(row + kernel_size), col:(col + kernel_size)]
                    # Operasi konvolusi
                    summa_nrc = sum(local_region * self.kernel[nkc]) + self.bias[nkc]
                    # Operasi aktivasi
                    cl[nkc, row, col] = relu_activation(summa_nrc)
                    # Operasi turunan aktivasi
                    d_cl_summa[nkc, row, col] = derivative_relu(cl[nkc, row, col])
                    d_summa_pl[0:n_kernel_prev, row:(row + kernel_size), col:(col + kernel_size),nkc ,row, col] = self.kernel[nkc]

        return cl, d_cl_summa, d_summa_pl

In [ ]:
class MaxPool:
    """
    Kelas untuk membuat pooling layer dengan operasi max pooling. Jumlah citra input dan output sama.

    Attribute:

    pool_size: int
        Ukuran pooling yang digunakan, bernilai default=2 yang berarti array 2 x 2.

    """

    def __init__(self, pool_size=2):
        self.pool_size = pool_size

    def forwardpass(self, cl):
        """
        Fungsi untuk pooling layer melakukan forwardpass.

        :param cl: Hasil convolutional layer sebelumnya.
        :return: Hasil operasi pooling dan lokasi indeks dari operasi pooling.
        """

        # Mendapatkan jumlah kernel dan ukuran kernel dari dimensi output conv layer sebelumnya
        n_kernel, cl_size, _ = cl.shape
        pl_size = int(cl_size / self.pool_size)

        pl = zeros((n_kernel, pl_size, pl_size), dtype=float64)
        index_loc = zeros((n_kernel, pl_size, pl_size), dtype=(int64, 2))

        for nk in range(n_kernel):
            for row in range(pl_size):
                for col in range(pl_size):
                    local_region = cl[nk,(2 * row):(2 * row + 2),(2 * col):(2 * col + 2)]
                    pl[nk, row, col] = max(local_region)
                    local_index = unravel_index(argmax(local_region), local_region.shape)
                    index_loc[nk, row, col] = [2 * row + local_index[0], 2 * col + local_index[1]]

        return pl, index_loc

In [ ]:
class ConvNet:
    """
    Kelas untuk membuat arsitektur ConvNet.
    """

    def __init__(self, n_kernel_1, n_kernel_2, n_kernel_3):
        # Ukuran kernel (width dan height sama, diambil salah satu
        KERNEL_SIZE = 5
        # Batas minimum
        MIN = -0.1
        # Batas maksimum
        MAX = 0.1
        # Banyak piksel (pl width x pl height) setelah flatten di fully connected layer
        FLATTEN_PIXELS = 16
        N_OUTPUTS = 2

        # Inisialisasi kernel dan bias convolution layer 1
        self.kernel_1 = random.uniform(low=MIN, high=MAX, size=(n_kernel_1, KERNEL_SIZE, KERNEL_SIZE))
        self.bias_1 = random.uniform(low=MIN, high=MAX, size=n_kernel_1)

        # Inisialisasi kernel dan bias convolution layer 2
        self.kernel_2 = random.uniform(low=MIN, high=MAX, size=(n_kernel_2, n_kernel_1, KERNEL_SIZE, KERNEL_SIZE))
        self.bias_2 = random.uniform(low=MIN, high=MAX, size=n_kernel_2)

        # Inisialisasi kernel dan bias convolution layer 3
        self.kernel_3 = random.uniform(low=MIN, high=MAX, size=(n_kernel_3, n_kernel_2, KERNEL_SIZE, KERNEL_SIZE))
        self.bias_3 = random.uniform(low=MIN, high=MAX, size=n_kernel_3)

        # Inisialisasi weight(bobot) dan bias fully connected layer
        self.weight = random.uniform(low=MIN, high=MAX, size=(N_OUTPUTS, n_kernel_3 * FLATTEN_PIXELS))
        self.bias = random.uniform(low=MIN, high=MAX, size=N_OUTPUTS)

    def forwardpass(self, image):
        """
        Fungsi untuk melakukan forwardpass.

        :param image: citra input.
        :return: citra dengan n-dimesional.

        """

        # Convolution layer 1
        conv_layer_1 = Conv2D(self.kernel_1, self.bias_1, padding=0, stride=1)
        cl1, d_cl1_sum1 = conv_layer_1.forwardpass(image)

        # Pooling layer 1
        pool_layer_1 = MaxPool(pool_size=2)
        pl1, index_loc1 = pool_layer_1.forwardpass(cl1)

        # Convolution layer 2
        conv_layer_2 = Conv3D(self.kernel_2, self.bias_2, padding=0, stride=1)
        cl2, d_cl2_sum2, d_sum2_pl1 = conv_layer_2.forwardpass(pl1)

        # Pooling layer 2
        pool_layer_2 = MaxPool(pool_size=2)
        pl2, index_loc2 = pool_layer_2.forwardpass(cl2)

        # Convolution layer 3
        conv_layer_3 = Conv3D(self.kernel_3, self.bias_3, padding=0, stride=1)
        cl3, d_cl3_sum3, d_sum3_pl2 = conv_layer_3.forwardpass(pl2)

        # Pooling layer 3
        pool_layer_3 = MaxPool(pool_size=2)
        pl3, index_loc3 = pool_layer_3.forwardpass(cl3)

        # Fully Connected layer
        # Merubah jadi 1 dimensi (n, dim1, dim2) -> n images (dim1)
        flattens = pl3.flatten()
        # Penjumlahah dari perkalian pembobotan
        summa = dot(self.weight, flattens) + self.bias
        # Class 0 dan 1 , outputs [0, 1]
        outputs = softmax_activation(summa)

        return flattens, outputs

    def training(self, images_training, labels_training, n_epoch, learn_rate):
        """
        Fungsi untuk melakukan pelatihan data.

        :param images_training: data citra latih.
        :param labels_training: data label dari citra latih.
        :param n_epoch: jumlah epoch.
        :param learn_rate: nilai learning rate.
        :return: epoch, rata-rata loss, dan akurasi.

        """

        n_kernel_1 = self.kernel_1.shape[0]
        n_kernel_2 = self.kernel_2.shape[0]
        n_kernel_3 = self.kernel_3.shape[0]

        train_epochs = []
        train_average_losses = []
        train_accuracies = []

        # Permutation - copy then shuffle
        len_perm = random.permutation(len(images_training))
        images = images_training[len_perm]
        labels = labels_training[len_perm]

        print("Pelatihan data: ")

        for epoch in range(n_epoch):

            train_losses = []
            correct_labels = 0

            progressbar_image = tqdm(total=len(images))

            for image, label in zip(images, labels):
                # Convolution layer 1
                conv_layer_1 = Conv2D(self.kernel_1, self.bias_1, padding=0, stride=1)
                cl1, d_cl1_sum1 = conv_layer_1.forwardpass(image)

                # Polling layer 1
                pool_layer_1 = MaxPool(pool_size=2)
                pl1, index_loc1 = pool_layer_1.forwardpass(cl1)

                # Convolution layer 2
                conv_layer_2 = Conv3D(self.kernel_2, self.bias_2, padding=0, stride=1)
                cl2, d_cl2_sum2, d_sum2_pl1 = conv_layer_2.forwardpass(pl1)

                # Pooling layer 2
                pool_layer_2 = MaxPool(pool_size=2)
                pl2, index_loc2 = pool_layer_2.forwardpass(cl2)

                # Convolution layer 3
                conv_layer_3 = Conv3D(self.kernel_3, self.bias_3, padding=0, stride=1)
                cl3, d_cl3_sum3, d_sum3_pl2 = conv_layer_3.forwardpass(pl2)

                # Pooling layer 3
                pool_layer_3 = MaxPool(pool_size=2)
                pl3, index_loc3 = pool_layer_3.forwardpass(cl3)

                # Fully connected layer
                flattens = pl3.flatten()
                weighted = dot(self.weight, flattens) + self.bias
                outputs = softmax_activation(weighted)

                train_losses.append(cross_entropy(outputs[label]))
                correct_labels += 1 if argmax(outputs) == label else 0

                # Backpropagation
                # Gradient softmax - fc layer
                dl_sum = copy(outputs)
                dl_sum[label] = outputs[label] - 1

                # Gradient bias - fc layer
                dl_bias = copy(dl_sum)

                # Gradient weight - fc layer
                dl_weight = zeros(outputs.shape + flattens.shape, dtype=float64)
                for i in range(outputs.shape[0]):
                    dl_weight[i,:] = outputs[i] * flattens
                dl_weight[label,:] = (outputs[label] - 1) * flattens

                dl_flattens = zeros(flattens.shape, dtype=float64)
                for j in range(flattens.shape[0]):
                    dl_flattens[j] = sum(dl_sum * self.weight[:,j])

                # Gradient output pooling layer 3
                dl_pl3 = dl_flattens.reshape(pl3.shape)

                # Gradient output convolution layer 3
                dl_cl3 = zeros(cl3.shape, dtype=float64)
                for nk in range(pl3.shape[0]):
                    for row in range(pl3.shape[1]):
                        for col in range(pl3.shape[2]):
                            u_max, v_max = index_loc3[nk,row,col]
                            dl_cl3[nk, u_max, v_max] = dl_pl3[nk,row,col]

                # Gradient output convolution layer 3
                dl_sum3 = dl_cl3 * d_cl3_sum3

                # Gradient bias_3 - convolution layer 3
                dl_bias_3 = zeros(self.kernel_3.shape[0], dtype=float64)

                # Gradient kernel_3 - convolution layer 3
                dl_kernel_3 = zeros(self.kernel_3.shape, dtype=float64)
                for nkc in range(self.kernel_3.shape[0]):
                    dl_bias_3[nkc] = sum(dl_sum3[nkc])
                    for nkp in range(self.kernel_3.shape[1]):
                        for row in range(self.kernel_3.shape[2]):
                            for col in range(self.kernel_3.shape[3]):
                                dl_kernel_3[nkc,nkp,row,col] = sum(dl_sum3[nkc] * pl2[nkp][row:(row+cl3.shape[1]), col:(col+cl3.shape[2])])

                # Gradient output pooling layer 2
                dl_pl2 = zeros(pl2.shape, dtype=float64)
                for ni in range(pl2.shape[0]):
                    for row in range(pl2.shape[1]):
                        for col in range(pl2.shape[2]):
                            dl_pl2[ni,row,col] = sum(dl_sum3 * d_sum3_pl2[ni,row,col])

                # Gradient output convolution layer 2
                dl_cl2 = zeros(cl2.shape, dtype=float64)
                for nk in range(pl2.shape[0]):
                    for row in range(pl2.shape[1]):
                        for col in range(pl2.shape[2]):
                            u_max, v_max = index_loc2[nk, row, col]
                            dl_cl2[nk, u_max, v_max] = dl_pl2[nk,row,col]

                # Gradient output convolution layer 2
                dl_sum2 = dl_cl2 * d_cl2_sum2

                # Gradient bias_2 - convolution layer 2
                dl_bias_2 = zeros(self.kernel_2.shape[0], dtype=float64)

                # Gradient kernel_2 - convolution layer 2
                dl_kernel_2 = zeros(self.kernel_2.shape, dtype=float64)
                for nkc in range(self.kernel_2.shape[0]):
                    dl_bias_2[nkc] = sum(dl_sum2[nkc])
                    for nkp in range(self.kernel_2.shape[1]):
                        for row in range(self.kernel_2.shape[2]):
                            for col in range(self.kernel_2.shape[3]):
                                dl_kernel_2[nkc,nkp,row,col] = sum(dl_sum2[nkc] * pl1[nkp][row:(row+cl2.shape[1]), col:(col+cl2.shape[2])])

                # Gradient output pooling layer 1
                dl_pl1 = zeros(pl1.shape, dtype=float64)
                for ni in range(pl1.shape[0]):
                    for row in range(pl1.shape[1]):
                        for col in range(pl1.shape[2]):
                            dl_pl1[ni,row,col] = sum(dl_sum2 * d_sum2_pl1[ni,row,col])

                # Gradient output convolution layer 1
                dl_cl1 = zeros(cl1.shape, dtype=float64)
                for nk in range(pl1.shape[0]):
                    for row in range(pl1.shape[1]):
                        for col in range(pl1.shape[2]):
                            i_max, j_max = index_loc1[nk,row,col]
                            dl_cl1[nk,i_max,j_max] = dl_pl1[nk,row,col]

                dl_sum1 = dl_cl1 * d_cl1_sum1

                # Gradient bias_1 - convolution layer 1
                dl_bias_1 = zeros(self.kernel_1.shape[0], dtype=float64)

                # gradient kernel_1 - convolution layer 1
                dl_kernel_1 = zeros(self.kernel_1.shape, dtype=float64)
                for nk in range(self.kernel_1.shape[0]):
                    dl_bias_1[nk] = sum(dl_sum1[nk])
                    for row in range(self.kernel_1.shape[1]):
                        for col in range(self.kernel_1.shape[2]):
                            dl_kernel_1[nk,row,col] = sum(dl_sum1[nk] * image[row:(row+cl1.shape[1]), col:(col+cl1.shape[2])])

                # Update parameters: kernel, bias, weight
                self.kernel_1 = self.kernel_1 - learn_rate * dl_kernel_1
                self.bias_1 = self.bias_1 - learn_rate * dl_bias_1
                self.kernel_2 = self.kernel_2 - learn_rate * dl_kernel_2
                self.bias_2 = self.bias_2 - learn_rate * dl_bias_2
                self.kernel_3 = self.kernel_3 - learn_rate * dl_kernel_3
                self.bias_3 = self.bias_3 - learn_rate * dl_bias_3
                self.weight = self.weight - learn_rate * dl_weight
                self.bias = self.bias - learn_rate * dl_bias

                progressbar_image.update()

            progressbar_image.close()

            train_epochs.append(epoch+1)

            # Hitung loss
            train_losses = array(train_losses)
            train_average_losses.append(train_losses.mean())

            # Hitung akurasi
            accuracy = correct_labels/len(labels) * 100
            train_accuracies.append(accuracy)

            print("Epoch ke-{}: rata-rata loss {}, akurasi {:02.2f}%.".format(epoch+1, train_losses.mean(), accuracy))

        return (train_epochs, train_average_losses, train_accuracies)

    def testing(self, images_testing, labels_testing):
        """
        Fungsi untuk melakukan pengujian data.

        :param images_testing: data citra uji.
        :param labels_testing: data label dari citra uji.
        :return: 

        """

        len_perm = random.permutation(len(images_testing))

        images = images_testing[len_perm]
        labels = labels_testing[len_perm]

        test_loss_values = []
        test_average_loss = []
        correct_labels = 0
        output = []
        result_class =[]

        counter_img = 0

        print("Pengujian data: ")

        for image, label in zip(images, labels):
            flattens, outputs = self.forwardpass(image)
            test_loss_values.append(cross_entropy(outputs[label]))
            # Menyimpan nilai outputs
            output.append(outputs)

            identify_image = argmax(outputs)
            result_class.append(identify_image)

            counter_img += 1
            print("Data ke-{}: kelas sebenarnya {}, diidentifikasi sebagai kelas {}, dengan hasil {}.".format(counter_img, label, identify_image , str(outputs)))

        # Hitung loss
        test_loss_values = array(test_loss_values)
        test_average_loss.append(test_loss_values.mean())

        saved_outputs = array(output)
        actual_class = array(labels)
        result_class = array(result_class)

        print("Nilai rata-rata loss {}.".format(test_loss_values.mean()))

        return test_average_loss, saved_outputs, actual_class, result_class

In [ ]:
numpy.set_printoptions(edgeitems=1000, linewidth=100000)

convnet_d_model = ConvNet(6, 6, 6)

# Pelatihan data
mfid_images_training = load("data/train_mfid_images.npy")
images_training = (mfid_images_training/255)-0.5

labels_training = load("data/train_mfid_labels.npy")

# Pengujian data
mfid_image_testing = load("data/test_mfid_images.npy")
images_testing = (mfid_image_testing/255)-0.5
labels_testing = load("data/test_mfid_labels.npy")

# Pelatihan model
train_d_model = convnet_d_model.training(images_training, labels_training, 15, 0.005)
train_epochs = train_d_model[0]
train_average_losses = train_d_model[1]
train_accuracies = train_d_model[2]

# Pengujian model
test_d_model = convnet_d_model.testing(images_testing, labels_testing)
test_average_loss = test_d_model[0]
outputs = test_d_model[1]
actual_class = test_d_model[2]
result_class = test_d_model[3]

# Simpan model setelah dilatih
with open("trained-model/d_model.bin", "wb") as file:
    pickle.dump(convnet_d_model, file)

# Plot diagram nilai loss
figure = plot.figure()
plot.subplots_adjust(hspace=0.5)

loss_graph = figure.add_subplot(2, 1, 1, ylabel="Train loss", xlabel="Epoch")
loss_graph.plot(train_epochs, train_average_losses, label="Average loss", color="red")
loss_graph.legend(loc="center")

# Diagram akurasi selama pelatihan data
accuracy_graph = figure.add_subplot(2, 1, 2, ylabel="Train accuracy", xlabel="Epoch")
accuracy_graph.plot(train_epochs, train_accuracies, label="Accuracy", color="green")
accuracy_graph.legend(loc="center")

In [ ]:
CM_ModD = ConfusionMatrix(actual_class, result_class)

CM_ModD.display_table()

print("Accuracy: {:02.8f}".format(CM_ModD.accuracy()))
print("Precision: {:02.8f}".format(CM_ModD.precision()))
print("Recall: {:02.8f}".format(CM_ModD.recall()))
print("F1-Score: {:02.8f}".format(CM_ModD.f1_score()))

CM_ModD.model_performance_graph()